<a href="https://colab.research.google.com/github/IBM-EPBL/IBM-Project-49171-1660816515/blob/main/Collecting_and_preprocessing_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [ ]:
df=pd.read_csv("Data/autos.csv",header=0,sep=',',encoding='latin',)

In [ ]:
df.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [ ]:
print(df.seller.value_counts()) 
df[df.seller != 'gewerblich'] 
df=df.drop('seller',1)
print(df.offerType.value_counts()) 
df[df.offerType != 'Gesuch'] 
df=df.drop ('offerType',1)

privat        371525
gewerblich         3
Name: seller, dtype: int64


C:\Users\aswin\AppData\Local\Temp\ipykernel_6164\3908850996.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop('seller',1)


Angebot    371516
Gesuch         12
Name: offerType, dtype: int64


C:\Users\aswin\AppData\Local\Temp\ipykernel_6164\3908850996.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop ('offerType',1)


In [ ]:
print(df.shape) 
df = df[(df.powerPS > 50) & (df.powerPS <900)] 
print(df.shape) 
print(df.shape) 
df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)] 
print(df.shape)

(371528, 18)
(319709, 18)
(319709, 18)
(309171, 18)


In [ ]:
df.drop(['name', 'abtest', 'dateCrawled', 'nrOfPictures', 'lastSeen','postalCode', 'dateCreated'], axis='columns', inplace=True)

In [ ]:
new_df = df.copy()

In [ ]:
df.columns

Index(['price', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS',
       'model', 'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage'],
      dtype='object')

In [ ]:
new_df.columns

Index(['price', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS',
       'model', 'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage'],
      dtype='object')

In [ ]:
new_df = new_df.drop_duplicates(['price', 'vehicleType', 'yearOfRegistration','gearbox', 'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType' , 'notRepairedDamage'])


In [ ]:
new_df.gearbox.replace(('manuell', 'automatik'), ('manual', 'automatic'), inplace=True) 
new_df.fuelType.replace(('benzin', 'andere', 'elektro'), ('petrol', 'others', 'electric'), inplace=True)
new_df.vehicleType.replace(('kleinwagen', 'cabrio', 'kombi', 'andere'),('small car', 'convertible', 'combination', 'others'), inplace=True) 
new_df.notRepairedDamage.replace(('ja', 'nein'), ('Yes', 'No'), inplace=True)

In [ ]:
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]
new_df['notRepairedDamage'].fillna (value='not-declared', inplace=True) 
new_df['fuelType'].fillna(value='not-declared', inplace=True) 
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna(value='not-declared', inplace=True) 
new_df['model'].fillna(value='not-declared', inplace=True)

In [ ]:
new_df.to_csv("autos_preprocessed.csv")

In [ ]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']
mapper = {}
for i in labels:
  mapper[i] = LabelEncoder() 
  mapper[i].fit(new_df[i]) 
  tr = mapper[i].transform(new_df[i])
  np.save(str('classes'+i+'.npy'), mapper[i].classes_)
  print(i,":", mapper[i])
  new_df.loc[:, i + '_labels'] = pd. Series (tr, index=new_df.index)

labeled = new_df[ ['price','yearOfRegistration' , 'powerPS' ,'kilometer' , 'monthOfRegistration']+ [x+"_labels" for x in labels]]
print(labeled.columns)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [ ]:
Y=labeled.iloc[:,0].values
X=labeled.iloc[:,1:].values
Y=Y.reshape(-1,1)

In [1]:
from sklearn.model_selection import cross_val_score, train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 3)

NameError: ignored